# Conversation Summary Buffer
---
`ConversationSummaryBuffer` combina las dos ideas. Mantiene un buffer de interacciones recientes en la memoria, pero en lugar de simplemente eliminar por completo las interacciones antiguas, las compila en un resumen y usa ambas. Utiliza la longitud del token en lugar del número de interacciones para determinar cuándo eliminar las interacciones.

Primero veamos cómo usar las utilidades.

### Usando la "memoria" con un LLM
---

In [2]:
from dotenv import load_dotenv, find_dotenv
from langchain.memory import ConversationSummaryBufferMemory
from langchain.llms import OpenAI

load_dotenv(find_dotenv())

True

In [3]:
# llm
llm = OpenAI()

In [4]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=10,
)

memory.save_context(
    inputs={"input":"hi"},
    outputs={"output": "whats up"}
)

memory.save_context(
    inputs={"input": "not much you"},
    outputs={"output": "not much"}
)

Veamos el contenido de la memoria, emplearemos el metodo `load_memory_variables`

In [5]:
memory.load_memory_variables({})

{'history': 'System: \nThe human greets the AI and the AI responds by asking what is up.\nHuman: not much you\nAI: not much'}

También podemos obtener el historial como una lista de mensajes (esto es útil si lo estás usando con un modelo de chat).

In [6]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=10,
    return_messages=True,
)

memory.save_context(
    inputs={"input":"hi"},
    outputs={"output": "whats up"}
)

memory.save_context(
    inputs={"input": "not much you"},
    outputs={"output": "not much"}
)

De igual forma, veamos el contenido

In [7]:
memory.load_memory_variables({})

{'history': [SystemMessage(content="\nThe human greets the AI and the AI responds by asking what's up.", additional_kwargs={}),
  HumanMessage(content='not much you', additional_kwargs={}, example=False),
  AIMessage(content='not much', additional_kwargs={}, example=False)]}


También podemos utilizar el método `predict_new_summary` directamente.

In [8]:
messages = memory.chat_memory.messages
previous_sumamry = ""
memory.predict_new_summary(messages, previous_sumamry)

'\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good. They then ask each other how they are doing, with both responding that not much is happening.'

## Usando en un "chain"
---

Veamos un ejemplo, configurando nuevamente `verbose=True` para que podamos ver el mensaje.

In [9]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(
    llm=OpenAI(),
    max_token_limit=40,
)

conversation_with_summary = ConversationChain(
    llm=llm,
    # We set a very low max_token_limit for the purposes of testing
    memory=memory,
    verbose=True,
)

# Lets make a test
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


" Hi there! I'm doing great. I'm currently working on an artificial intelligence project that involves natural language processing. I'm trying to get better at understanding how humans communicate with each other. How about you?"

In [10]:
conversation_with_summary.predict(input="Just working on writing some documentation!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human greets the AI and the AI responds, stating that it is working on an artificial intelligence project involving natural language processing to better understand how humans communicate.
Human: Just working on writing some documentation!
AI:

> Finished chain.


' Ah, I see. That sounds like a very interesting and important project. Natural language processing is a fascinating field of study. Have you been working on this project for long?'

In [11]:
# We can see here that there is a summary of the conversation and then some previous interactions
conversation_with_summary.predict(input="For LangChain! Have you heard of it?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human greets the AI and the AI responds, stating that it is working on an artificial intelligence project involving natural language processing to better understand how humans communicate. The human reveals that they are just working on writing some documentation.
AI:  Ah, I see. That sounds like a very interesting and important project. Natural language processing is a fascinating field of study. Have you been working on this project for long?
Human: For LangChain! Have you heard of it?
AI:

> Finished chain.


' Yes, I have heard of LangChain. I understand that it is a platform for language learning, which aims to make it easier for people to learn new languages. Is that correct?'

In [12]:
# We can see here that the summary and the buffer are updated
conversation_with_summary.predict(
    input="Haha nope, although a lot of people confuse it for that"
)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human greets the AI and the AI responds, stating that it is working on an artificial intelligence project involving natural language processing to better understand how humans communicate. The human reveals that they are just working on writing some documentation for LangChain, to which the AI expresses its interest in natural language processing and inquires as to how long the human has been working on the project.
AI:  Yes, I have heard of LangChain. I understand that it is a platform for language learning, which aims to make it easier for people to learn new languages. Is that correct?
Human: Haha nope, although a lot of people confuse it for that
AI:



' Oh, I see. So, what is LangChain then?'